In [123]:
import re
from datetime import datetime,timedelta

In [124]:
with open('jason.1.input') as f:
    data = [ line.strip().rstrip() for line in f.readlines()]
    
base_time = 60


In [113]:
raw_test_data = '''Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.'''

test_data = [ line.strip().rstrip() for line in raw_test_data.split('\n')]
data = test_data
base_time = 0
print(len(test_data))
print('\n'.join(test_data))



7
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.


In [125]:
print(base_time + ord('A'.lower()) - 96)
print(base_time + ord('Z'.lower()) - 96)

61
86


In [126]:
class node:
    def __init__(self,val):
        self.val = val
        self.after=[]
        self.before=[]
        self._complete = 0
        self.work_duration = timedelta(seconds=(base_time + ord(val.lower()) - 96 - 1))
        self.locked = 0
        
    def __lt__(self, value):
        if isinstance(value,node):
            return self.val<=value.val
        else:
            return self.val<=value
    
    def __eq__(self,other):
        return self.val == other
        
    def __repr__(self):
        return self.val
    
    def __str__(self):
        return self.val
    
    def step_ready(self):
        return (all(e._complete for e in self.before) or not len(self.before)) and not self._complete and not self.locked
    
    def print(self):
        print('Val:',self.val,'Complete:',self._complete,'before:',self.before,'after:',self.after)
        
    def complete_step(self):
        self._complete = 1
        
    def is_complete(self):
        return self._complete
    
    def set_lock(self):
        self.locked = 1
        
    def is_locked(self):
        return self.locked
    
    def add_after(self,ele):
        self.after.append(ele)
        self.after.sort(key=lambda x:x.val)
        
    def add_before(self,ele):
        self.before.append(ele)
        self.before.sort(key=lambda x:x.val)
        

In [127]:
nodes = {}
# Step I must be finished before step Q can begin.
r = 'Step (\w) must be finished before step (\w) can begin.'
for line in data:
    before, after = re.match(r,line).groups()
#     print(line)
    if before not in nodes:
        nodes[before] = node(before)
    if after not in nodes:
        nodes[after] = node(after)

    nodes[after].add_before(nodes[before])
    nodes[before].add_after(nodes[after])


In [128]:

def all_complete():
    return all(n.is_complete() for n in nodes.values())


order = ''
while not all_complete():
    next_q = []
    for k,n in nodes.items():
        if n.step_ready():
            next_q.append(n)

    next_q.sort()
    if len(next_q):
        n=next_q.pop(0)
        order+=n.val
        n.complete_step()
        
print('order:',order)

for k in nodes:
    nodes[k]._complete=0


order: IBJTUWGFKDNVEYAHOMPCQRLSZX


In [134]:
class worker:
    
    def __init__(self,name):
        self.name = name
        self.current_node = None
        self.ttc = datetime.now()
    
    def assign(self, now, node):
        self.current_node = node
        self.current_node.set_lock()
        self.ttc = now + node.work_duration

    def get_current_job(self):
        return self.current_node.val if self.current_node is not None else '.'
        
    def is_complete(self, now):
        if now>=self.ttc and self.current_node is not None:
            self.current_node.complete_step()
#             self.ttc = None
            self.last_job = self.current_node
            self.current_node = None
            return self.last_job
        return None
        
    def is_free(self,):
        return self.current_node == None
    

In [145]:

nodes = {}
# Step I must be finished before step Q can begin.
r = 'Step (\w) must be finished before step (\w) can begin.'
for line in data:
    before, after = re.match(r,line).groups()
    if before not in nodes:
        nodes[before] = node(before)
    if after not in nodes:
        nodes[after] = node(after)

    nodes[after].add_before(nodes[before])
    nodes[before].add_after(nodes[after])

def available_worker():
    return any(w.is_free() for w in workers)

def get_available_worker():
    for w in workers:
        if w.is_free():
            return w
        
def get_queue(nodes):
    next_q = []
    for k,n in nodes.items():
        if n.step_ready():
            next_q.append(n)
    return sorted(next_q)

order = ''
n_workers = 5
workers = [ worker('worker_{}'.format(i)) for i in range(n_workers)]
start = datetime.now()
now = datetime.now()
dt = timedelta(seconds=1)
end = now+timedelta(seconds=5000)

while not all_complete() and now<end:
    next_q = get_queue(nodes)
    
    for w in workers:
        ret = w.is_complete(now)
        if ret is not None:
            order+=ret.val
            ret.complete_step()
            next_q = get_queue(nodes)
    now += dt
    while available_worker() and len(next_q):
        w = get_available_worker()
        n=next_q.pop(0)
        sorted(next_q)
        w.assign(now,n)
    
    
    print(now.strftime('%H:%M:%S'),'\t'.join(w.get_current_job() for w in workers),order)

print('order:',order)
print("Seconds needed:", int((now-start).total_seconds()-1))
for k in nodes:
    nodes[k]._complete=0

18:23:59 I	T	U	W	. 
18:24:00 I	T	U	W	. 
18:24:01 I	T	U	W	. 
18:24:02 I	T	U	W	. 
18:24:03 I	T	U	W	. 
18:24:04 I	T	U	W	. 
18:24:05 I	T	U	W	. 
18:24:06 I	T	U	W	. 
18:24:07 I	T	U	W	. 
18:24:08 I	T	U	W	. 
18:24:09 I	T	U	W	. 
18:24:10 I	T	U	W	. 
18:24:11 I	T	U	W	. 
18:24:12 I	T	U	W	. 
18:24:13 I	T	U	W	. 
18:24:14 I	T	U	W	. 
18:24:15 I	T	U	W	. 
18:24:16 I	T	U	W	. 
18:24:17 I	T	U	W	. 
18:24:18 I	T	U	W	. 
18:24:19 I	T	U	W	. 
18:24:20 I	T	U	W	. 
18:24:21 I	T	U	W	. 
18:24:22 I	T	U	W	. 
18:24:23 I	T	U	W	. 
18:24:24 I	T	U	W	. 
18:24:25 I	T	U	W	. 
18:24:26 I	T	U	W	. 
18:24:27 I	T	U	W	. 
18:24:28 I	T	U	W	. 
18:24:29 I	T	U	W	. 
18:24:30 I	T	U	W	. 
18:24:31 I	T	U	W	. 
18:24:32 I	T	U	W	. 
18:24:33 I	T	U	W	. 
18:24:34 I	T	U	W	. 
18:24:35 I	T	U	W	. 
18:24:36 I	T	U	W	. 
18:24:37 I	T	U	W	. 
18:24:38 I	T	U	W	. 
18:24:39 I	T	U	W	. 
18:24:40 I	T	U	W	. 
18:24:41 I	T	U	W	. 
18:24:42 I	T	U	W	. 
18:24:43 I	T	U	W	. 
18:24:44 I	T	U	W	. 
18:24:45 I	T	U	W	. 
18:24:46 I	T	U	W	. 
18:24:47 I	T	U	W	. 
18:24:48 I	T	U	W	. 
